## Siamese

In [1]:
import os
os.chdir('../../src/')

In [2]:
import torch
import torch.utils.data as data
from PIL import Image, ImageOps
import pickle
import numpy as np
from torchvision import transforms
import json
from credential_classifier.bit_pytorch.dataloader import HybridLoader
from credential_classifier.bit_pytorch.models import KNOWN_MODELS
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"]="0"

- Define dataloader

In [3]:
val_set = HybridLoader(img_folder='../datasets/val_merge_imgs',
                         annot_path='../datasets/val_merge_coords.txt')

valid_loader = torch.utils.data.DataLoader(
  val_set, batch_size=1, shuffle=False, pin_memory=True, drop_last=False)

In [4]:
len(valid_loader)

3731

- Accuracy function

In [15]:
def compute_acc(dataloader, model, device):
    correct = 0
    total = 0

    for b, (x, y) in tqdm(enumerate(dataloader)):
        with torch.no_grad():
            x = x.to(device, non_blocking=True, dtype=torch.float)
            y = y.to(device, non_blocking=True)
            logits = model(x)
            pred_cls = torch.argmax(logits, dim=1)

            correct += torch.sum(torch.eq(pred_cls, y)).item()
            total += y.shape[0]
            
    print('Accuracy after changing relu function: {:.2f}'.format(correct/total))    
    return correct/total

- Load model (original)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1V2"](head_size=2, zero_head=True)

# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/credential_classifier/output/hybrid/hybrid_lr0.005/BiT-M-R50x1V2_0.005.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)

model.to(device)
model.eval()

ResNetV2Coord(
  (root): Sequential(
    (conv): StdConv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05

In [16]:
compute_acc(valid_loader, model, device)

3731it [11:43,  5.31it/s]

Accuracy after changing relu function: 0.95


0.9471991423210936

## Load model (change activation function)

In [8]:
import torch
import torch.nn.functional as F
from torch import nn

In [9]:
class QuantizeRelu(nn.Module):
    def __init__(self, step_size = 0.01):
        super().__init__()
        self.step_size = step_size

    def forward(self, x):
        mask = torch.ge(x, 0).bool() # mask for positive values
        quantize = torch.ones_like(x) * self.step_size
        out = torch.mul(torch.floor(torch.div(x, quantize)), self.step_size) # quantize by step_size
        out = torch.mul(out, mask) # zero-out negative values
        out = torch.abs(out) # remove sign
        return out

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model
model = KNOWN_MODELS["BiT-M-R50x1V2"](head_size=2, zero_head=True)
# Load weights
checkpoint = torch.load('/home/l/liny/ruofan/PhishIntention/src/credential_classifier/output/hybrid/hybrid_lr0.005/BiT-M-R50x1V2_0.005.pth.tar', 
                        map_location="cpu")["model"]

from collections import OrderedDict
new_state_dict = OrderedDict()
for k, v in checkpoint.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v

model.load_state_dict(new_state_dict)
# replace relu with defenselayer 
# model.body.block1.unit01.relu = QuantizeRelu()
# model.body.block1.unit02.relu = QuantizeRelu()
# model.body.block1.unit03.relu = QuantizeRelu()

# model.body.block2.unit01.relu = QuantizeRelu()
# model.body.block2.unit02.relu = QuantizeRelu()
# model.body.block2.unit03.relu = QuantizeRelu()
# model.body.block2.unit04.relu = QuantizeRelu()

# model.body.block3.unit01.relu = QuantizeRelu()
# model.body.block3.unit02.relu = QuantizeRelu()
# model.body.block3.unit03.relu = QuantizeRelu()
# model.body.block3.unit04.relu = QuantizeRelu()
# model.body.block3.unit05.relu = QuantizeRelu()
# model.body.block3.unit06.relu = QuantizeRelu()

model.body.block4.unit01.relu = QuantizeRelu()
model.body.block4.unit02.relu = QuantizeRelu()
model.body.block4.unit03.relu = QuantizeRelu()

model.to(device)
model.eval()

ResNetV2Coord(
  (root): Sequential(
    (conv): StdConv2d(8, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pad): ConstantPad2d(padding=(1, 1, 1, 1), value=0)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (body): Sequential(
    (block1): Sequential(
      (unit01): PreActBottleneck(
        (gn1): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv1): StdConv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (gn2): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv2): StdConv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (gn3): GroupNorm(32, 64, eps=1e-05, affine=True)
        (conv3): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (relu): ReLU(inplace=True)
        (downsample): StdConv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      )
      (unit02): PreActBottleneck(
        (gn1): GroupNorm(32, 256, eps=1e-05

In [19]:
# compute_acc(valid_loader, model, device)

Accuracy after changing relu function: 0.93


0.9345417925478349

## Attack

In [ ]:
from adv_attack.attack.Attack import *

criterion = nn.CrossEntropyLoss()
check = adversarial_attack(method='cw', model=model, dataloader=valid_loader, 
                           device=device, num_classes=2, save_data=True)

acc, _ = check.batch_attack()

64it [2:06:54, 82.46s/it]

63Learning Progress : 100.00 % 
Test Accuracy = 0.96875


65it [2:08:32, 87.27s/it]

64Learning Progress : 100.00 % 
Test Accuracy = 0.9692307692307692


66it [2:10:33, 97.26s/it]

...

732
Test Accuracy = 0.9549795361527967


734it [22:29:20, 61.38s/it]

733


735it [22:30:48, 69.19s/it]

734earning Progress : 100.00 % 
Test Accuracy = 0.9537414965986395


In [12]:
acc

0.9471991423210936

## BPDA

- With defense

In [10]:
from advertorch.bpda import BPDAWrapper
from advertorch.attacks import LinfPGDAttack

In [25]:
# original model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = KNOWN_MODELS["BiT-M-R50x1"](head_size=180, zero_head=True)
model.load_state_dict(torch.load('/home/l/liny/ruofan/phishpedia/siamese/checkpoints/resnetv2_rgb.pth', map_location=device))
model.to(device)

# BPDA approximation of gradients
defense_layer = BPDAWrapper(QuantizeRelu())

# replace relu with defenselayer 
model.body.block1.unit01.relu = defense_layer
model.body.block1.unit02.relu = defense_layer
model.body.block1.unit03.relu = defense_layer

model.body.block2.unit01.relu = defense_layer
model.body.block2.unit02.relu = defense_layer
model.body.block2.unit03.relu = defense_layer
model.body.block2.unit04.relu = defense_layer

model.body.block3.unit01.relu = defense_layer
model.body.block3.unit02.relu = defense_layer
model.body.block3.unit03.relu = defense_layer
model.body.block3.unit04.relu = defense_layer
model.body.block3.unit05.relu = defense_layer
model.body.block3.unit06.relu = defense_layer

model.body.block4.unit01.relu = defense_layer
model.body.block4.unit02.relu = defense_layer
model.body.block4.unit03.relu = defense_layer

bpda_adversary = LinfPGDAttack(
    model, loss_fn=nn.CrossEntropyLoss(reduction="sum"), eps=0.05,
    nb_iter=100, eps_iter=0.005, rand_init=True, clip_min=0.0, clip_max=1.0,
    targeted=False)

In [27]:
perturb_correct = 0
total = 0

for cln_data, true_label in tqdm(valid_loader):
    cln_data, true_label = cln_data.to(device), true_label.to(device)
    bpda_adv = bpda_adversary.perturb(cln_data, true_label)
    
    logits = model(bpda_adv)
    pred_cls = torch.argmax(logits, dim=1)
    perturb_correct += torch.sum(torch.eq(pred_cls, true_label)).item()
    total += len(true_label)
    
    print(perturb_correct/total)

 12%|█▎        | 1/8 [21:33<2:30:57, 1293.95s/it]

0.890625


 25%|██▌       | 2/8 [44:51<2:12:30, 1325.09s/it]

0.9140625


 38%|███▊      | 3/8 [1:08:51<1:53:17, 1359.44s/it]

...

0.921875


 88%|████████▊ | 7/8 [2:43:00<23:17, 1397.87s/it]

0.9274553571428571


100%|██████████| 8/8 [2:58:41<00:00, 1340.19s/it]

0.9274924471299094


In [28]:
perturb_correct/total

0.9274924471299094

In [24]:
perturb_correct/total

0.905337361530715

In [14]:
torch.randint(0, 277, (1,))

tensor([203])